# pytopomat Example Notebook
This notebook will show you how to
* Obtain symmetry operation eigenvalues with IrRep, Irvsp, and Vasp2Trace
* Compute topological invariants directly from output files.
* Submit workflows for high-throughput band topology calculations.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
test_dir = "test_files/"

## IrRep to compute irreducible representations (https://arxiv.org/abs/2009.01764)
Install irrep from: https://github.com/stepan-tsirkin/irrep

Following the instructions here: https://pypi.org/project/irrep/ 

Run a calculation with LWAVE = .TRUE.  
Make sure the crystal is given in a standard setting, e.g. by doing `phonopy --tolerance 0.01 --symmetry -c POSCAR`

**NOTE: The IrRep tool in the example below can also be substitued for Irvsp.

In [3]:
from pytopomat.irrep_caller import IrrepCaller, IrrepOutput

In [ ]:
# In a directory with OUTCAR, WAVECAR, and POSCAR; we'll get an error if the irrep script is not in the path
ic = IrrepCaller('.')
irrep_out = ic.output

In [5]:
# Take an irrep output that has been pre-computed for a spin-polarized system
irrep_out = IrrepOutput(test_dir + "CrO2_sp_irrep.txt")

/home/jason/Documents/DEVELOP/pytopomat/pytopomat/irrep_caller.py:221: UserWarning: Irrep output data has non-integer parity eigenvalues!
  "Irrep output data has non-integer parity eigenvalues!"


In [6]:
# The IrrepOutput instance contains parity eigenvalues for all the bands at the TRIM points
irrep_out.parity_eigenvals.keys()

dict_keys(['gamma', 'x', 'y', 'z', 's', 't', 'u', 'r'])

In [7]:
# Inspect the parity eigenvalues in the first 5 valence bands at the Gamma point
irrep_out.parity_eigenvals["gamma"]["inversion_eigenval"][:5]

[-1.0, -1.0, -1.0, -1.0, -1.0]

### Compute the topological invariants from irvsp output
The BandParity analyzer will try to find a gapped subspace of valence bands and assume that the lower-lying bands are topologically trivial.  
For systems with time-reversal and inversion symmetries, we can compute the $\mathbb{Z}_2$ invariant.    
For systems breaking time-reversal but preserving inversion we can still do some analyze using the parity eigenvalues.  
For 2D systems `compute_z2` will return the $\mathbb{Z}_2$ invariant.   
For 3D systems it will compute $\mathbb{Z}_2$ = ($v_0$; $v_1$, $v_2$, $v_3$)

In [8]:
from pytopomat.analyzer import BandParity

For inversion symmetric systems breaking time-reversal, the strong topological index $z_4$ as defined in Eq 1 of https://arxiv.org/abs/2003.00012.  
* $z_4 = 0$ -> trivial
* $z_4 = 1, 3$ -> Weyl semimetal phase (odd number of of Weyl points in half of the Brillouin zone)
* $z_4 = 2$ -> Axion insulator with quantized topological magneto-electric response

In [9]:
# We have to specify at least thethe Fermi level (E-fermi in VASP OUTCAR) from the calculation for irrep and irvsp, but not for vasp2trace
# Passing the total number of electrons (nelect) is highly recommended to ensure proper selection of occupied states 
bp = BandParity(irrep_out, spin_polarized=True, nelect=48, efermi=4.6476)

bp.compute_z4()

3.0

In [10]:
# An example with SOC and no spin-polarization
irrep_out = IrrepOutput(test_dir + "Na3Bi_irrep.txt")
bp = BandParity(irrep_out, spin_polarized=False, nelect=26, efermi=2.6723)

bp.compute_z2(tol=-1)  # Tolerance for energy differences between bands to define degeneracy; test the sensistivity of results to this parameter!

Only considering last 13 pairs of bands.


/home/jason/Documents/DEVELOP/pytopomat/pytopomat/irrep_caller.py:221: UserWarning: Irrep output data has non-integer parity eigenvalues!
  "Irrep output data has non-integer parity eigenvalues!"


array([1., 0., 0., 0.])

Note that with the default tol value above (-1), we use all occupied bands to compute the $\mathbb{Z}_2$ invariant for Na3Bi. 
If we adjust the tolerance and consider a different subspace of bands, the invariants may change.  
You should check the sensitivity of invariants with respect to the tolerance.

In [11]:
bp.compute_z2(tol=0.1)

Only considering last 1 pairs of bands.


array([0., 0., 0., 0.])

In [12]:
bp.compute_z2(tol=2)

Only considering last 3 pairs of bands.


array([1., 0., 0., 0.])

## Vasp2Trace Analysis
v2t has two versions, v1 for non-spin-polarized systems and v2 for spin-polarized systems.

In [13]:
from pytopomat.vasp2trace_caller import Vasp2TraceCaller, Vasp2Trace2Caller, Vasp2TraceOutput

In [14]:
vc = Vasp2TraceCaller('.')  # as before, we'll get a warning if we don't have the executables in the path
vc2 = Vasp2Trace2Caller('.')

RuntimeError: Vasp2TraceCaller requires vasp2trace to be in the path.Please follow the instructions at http://www.cryst.ehu.es/cgi-bin/cryst/programs/topological.pl.

In [15]:
out = Vasp2TraceOutput(test_dir + 'Bi2Se3_trace_soc.txt')  # load precomputed data

In [16]:
# Look at some of the info v2t gives us
print(out.num_occ_bands, '\n')
print(out.kvecs, '\n')  # TRIM points
for so in out.symm_ops:
    print(so)  # Here we have the identiy and inversion operations

144 

[[0.0, 0.0, 0.0], [0.0, 0.0, 0.5], [0.0, 0.5, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.0], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0], [0.5, 0.5, 0.5]] 

[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
[-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]


### Topological invariants from v2t
To process the output we need to format it as a dictionary that contains v2t output for each spin channel. Then we can do the same analysis as before.

In [17]:
import numpy as np

bp = BandParity({'up': out})

# Loop through to try a range of tolerance values
for tol in np.linspace(0.1, 0.15, 5):
    try:
        print(bp.compute_z2(tol=tol))
    except:
        pass

Only considering last 3 pairs of bands.
[1. 0. 0. 1.]
Only considering last 3 pairs of bands.
[1. 0. 0. 1.]
Only considering last 24 pairs of bands.
[0. 0. 0. 0.]
Only considering last 24 pairs of bands.
[0. 0. 0. 0.]
Only considering last 24 pairs of bands.
[0. 0. 0. 0.]


/home/jason/Documents/DEVELOP/pytopomat/pytopomat/analyzer.py:490: UserWarning: Number of electrons not provided. Will try and infer total from identity eigenvalues.
  "Number of electrons not provided. Will try and infer total from identity eigenvalues."


## Spin-polarization
Let's see what happens if we use v2t with spin-polarization.

In [18]:
u = Vasp2TraceOutput(test_dir + 'Bi2Se3_trace_up.txt')
d = Vasp2TraceOutput(test_dir + 'Bi2Se3_trace_dn.txt')
bp = BandParity({'up': u, 'down': d})
bp.compute_z4()

0.0

In [19]:
bp.screen_magnetic_parity()  # Quick summary of allowable properties

{'insulator': True,
 'semimetal_candidate': False,
 'polarization_bqhc': True,
 'magnetoelectric': True}

## Z2Pack
z2p requires an input directory with all VASP output files generated from a calc with some specific Wannier90 flags.  
The easiest way to do this is to run the wflow below, where this is all automated.  
If you had a calculation completed, you could use `Z2PackCaller` directly to run z2p.

In [20]:
from pytopomat.z2pack_caller import Z2PackCaller

In [ ]:
zc = Z2PackCaller(input_dir='input', surface='kx_0', vasp_cmd='srun vasp_ncl >& log')
zc.run(z2_settings={'pos_tol': 0.02})  # check z2p docs for explanation of optional convergence settings

In [21]:
from pytopomat.z2pack_caller import Z2Output
import z2pack

In [22]:
result = z2pack.io.load(test_dir+'res_1.json')  # load a pre-computed surface calculation
out = Z2Output(result, surface='kx_0')

The z2p output has topological invariants stored so we can just print them.

In [23]:
print(out.z2_invariant, out.chern_number)

0 -1.258173527141082e-06


## Workflows for high-throughput band topology
Generating wflows for topology calculations is easy with pytopomat.

In [ ]:
from pymatgen import Structure, Lattice

In [ ]:
bcc_bi = Structure.from_spacegroup("Im-3m", Lattice.cubic(3.453), ["Bi"], [[0, 0, 0]])

Vasp2Trace and irvsp wflows do band structure calculations at TRIM kpoints and compute the parity eigenvalues for obtaining topological invariants. They also generate "trace.txt" for non-spin-polarized calculations and "trace_up.txt" and "trace_dn.txt" files for spin-polarized calculations, which can be used to check the topological character of a material with Topological Quantum Chemistry.
https://www.cryst.ehu.es/cgi-bin/cryst/programs/topological.pl

To run the wflows in atomate, copy all the .yaml files from pytopomat/workflows into your atomate library, which in an anaconda environment is probably located here:  
~/.conda/envs/<my_anaconda_environment_name>/lib/python3.6/site-packages/atomate/vasp/workflows/base/library/

In [ ]:
from pytopomat.workflows.core import wf_vasp2trace_nonmagnetic, wf_vasp2trace_magnetic, wf_irvsp

wf1 = wf_vasp2trace_nonmagnetic(bcc_bi)

In [ ]:
bcc_bi.add_site_property("magmom", [5.0, 5.0])  # must have "magmom" site property for magnetic wflows
wf2 = wf_vasp2trace_magnetic(bcc_bi)

In [ ]:
wf3 = wf_irvsp(bcc_bi, magnetic=True, soc=False, v2t=True)  # Run a combined v2t and irvsp wflow

The Z2Pack wflow is a more general wflow that uses hybrid Wannier charge centers to compute Z2 indices and Chern numbers on inequivalent TRI planes in the 3D Brillouin zone.

In [ ]:
from pytopomat.workflows.core import Z2PackWF

wf4 = Z2PackWF(bcc_bi, symmetry_reduction=False).get_wf()  # symm reduction can be used in non-magnetic wflows to reduce the number of BZ surface calcs

In [ ]:
from fireworks import LaunchPad

# Define a LaunchPad for your database
lpad = LaunchPad(
    host="mongodb03.nersc.gov",
    name="",
    username="",
    password=""
)

for wf in [wf1, wf2, wf3, wf4]:
    lpad.add_wf(wf)